# 📓 TruLens Quickstart

In this quickstart you will create a RAG from scratch and learn how to log it and get feedback on an LLM response.

For evaluation, we will leverage the "hallucination triad" of groundedness, context relevance and answer relevance.

In [ ]:
# !pip install --pre trulens trulens-providers-openai trulens-connectors-snowflake chromadb openai

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ""

## Get Data

In this case, we'll just initialize some simple text in the notebook.

In [ ]:
uw_info = """
The University of Washington, founded in 1861 in Seattle, is a public research university
with over 45,000 students across three campuses in Seattle, Tacoma, and Bothell.
As the flagship institution of the six public universities in Washington state,
UW encompasses over 500 buildings and 20 million square feet of space,
including one of the largest library systems in the world.
"""

wsu_info = """
Washington State University, commonly known as WSU, founded in 1890, is a public research university in Pullman, Washington.
With multiple campuses across the state, it is the state's second largest institution of higher education.
WSU is known for its programs in veterinary medicine, agriculture, engineering, architecture, and pharmacy.
"""

seattle_info = """
Seattle, a city on Puget Sound in the Pacific Northwest, is surrounded by water, mountains and evergreen forests, and contains thousands of acres of parkland.
It's home to a large tech industry, with Microsoft and Amazon headquartered in its metropolitan area.
The futuristic Space Needle, a legacy of the 1962 World's Fair, is its most iconic landmark.
"""

starbucks_info = """
Starbucks Corporation is an American multinational chain of coffeehouses and roastery reserves headquartered in Seattle, Washington.
As the world's largest coffeehouse chain, Starbucks is seen to be the main representation of the United States' second wave of coffee culture.
"""

## Create Vector Store

Create a chromadb vector store in memory.

In [ ]:
import os

import chromadb
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

embedding_function = OpenAIEmbeddingFunction(
    api_key=os.environ.get("OPENAI_API_KEY"),
    model_name="text-embedding-ada-002",
)


chroma_client = chromadb.Client()
vector_store = chroma_client.get_or_create_collection(
    name="Washington", embedding_function=embedding_function
)

Populate the vector store.

In [ ]:
vector_store.add("uw_info", documents=uw_info)
vector_store.add("wsu_info", documents=wsu_info)
vector_store.add("seattle_info", documents=seattle_info)
vector_store.add("starbucks_info", documents=starbucks_info)

## Build RAG from scratch

Build a custom RAG from scratch, and add TruLens custom instrumentation.

In [ ]:
from trulens.connectors.snowflake import SnowflakeConnector
from trulens.core.app.custom import instrument
from trulens.core.session import TruSession

connector = SnowflakeConnector(
    account="",
    user="",
    password="",
    database_name="",
    schema_name="",
    warehouse="",
    role="",
)
# session = TruSession()
# session = Tru()
session = TruSession(connector=connector)

In [ ]:
from openai import OpenAI

oai_client = OpenAI()

In [ ]:
from openai import OpenAI

oai_client = OpenAI()


class RAG_from_scratch:
    @instrument
    def retrieve(self, query: str) -> list:
        """
        Retrieve relevant text from vector store.
        """
        results = vector_store.query(query_texts=query, n_results=4)
        # Flatten the list of lists into a single list
        return [doc for sublist in results["documents"] for doc in sublist]

    @instrument
    def generate_completion(self, query: str, context_str: list) -> str:
        """
        Generate answer from context.
        """
        completion = (
            oai_client.chat.completions.create(
                model="gpt-3.5-turbo",
                temperature=0,
                messages=[
                    {
                        "role": "user",
                        "content": f"We have provided context information below. \n"
                        f"---------------------\n"
                        f"{context_str}"
                        f"\n---------------------\n"
                        f"Given this information, please answer the question: {query}",
                    }
                ],
            )
            .choices[0]
            .message.content
        )
        return completion

    @instrument
    def query(self, query: str) -> str:
        context_str = self.retrieve(query)
        completion = self.generate_completion(query, context_str)
        return completion


rag = RAG_from_scratch()

## Set up feedback functions.

Here we'll use groundedness, answer relevance and context relevance to detect hallucination.

In [ ]:
import numpy as np
from trulens.core import Feedback
from trulens.core import Select
from trulens.providers.openai import OpenAI

provider = OpenAI(model_engine="gpt-4o")

# Define a groundedness feedback function
f_groundedness = (
    Feedback(
        provider.groundedness_measure_with_cot_reasons, name="Groundedness"
    )
    .on(Select.RecordCalls.retrieve.rets.collect())
    .on_output()
)
# Question/answer relevance between overall question and answer.
f_answer_relevance = (
    Feedback(provider.relevance_with_cot_reasons, name="Answer Relevance")
    .on_input()
    .on_output()
)

# Context relevance between question and each context chunk.
f_context_relevance = (
    Feedback(
        provider.context_relevance_with_cot_reasons, name="Context Relevance"
    )
    .on_input()
    .on(Select.RecordCalls.retrieve.rets[:])
    .aggregate(np.mean)  # choose a different aggregation method if you wish
)

## Construct the app
Wrap the custom RAG with TruCustomApp, add list of feedbacks for eval

In [ ]:
from trulens.core import TruCustomApp
from trulens.core.database.connector import DefaultDBConnector

# Expected API. Should run without error and create app_id from hash
tru_rag = TruCustomApp(
    rag,
    app_name="RAG",
    app_version="0.1",
    feedbacks=[f_groundedness, f_answer_relevance, f_context_relevance],
)

# Should run and assign a default app_version
tru_rag = TruCustomApp(
    rag,
    app_name="RAG",
    feedbacks=[f_groundedness, f_answer_relevance, f_context_relevance],
)

# Should run and assign app_id to app_name with deprecation warning
# tru_rag = TruCustomApp(
#     rag,
#     app_id="RAG",
#     app_version="0.1",
#     feedbacks=[f_groundedness, f_answer_relevance, f_context_relevance],
# )

# Should write to sqlite db
# tru_rag = TruCustomApp(
#     rag,
#     app_id="RAG",
#     app_name="RAG",
#     connector=DefaultDBConnector(),
#     app_version="0.1",
#     feedbacks=[f_groundedness, f_answer_relevance, f_context_relevance],
# )

In [ ]:
from trulens.core import TruSession

tru = TruSession()
assert tru.connector is connector

In [ ]:
from trulens.core import TruSession

# should raise warning that connector cannot be overridden once set
default_connector = DefaultDBConnector()
tru = TruSession(connector=default_connector)

## Run the app
Use `tru_rag` as a context manager for the custom RAG-from-scratch app.

In [ ]:
with tru_rag as recording:
    rag.query("When was the University of Washington founded?")

## Check results

We can view results in the leaderboard.

In [ ]:
session.get_leaderboard()

In [ ]:
from trulens.core.utils.trulens import get_feedback_result

last_record = recording.records[-1]
get_feedback_result(last_record, "Context Relevance")